<a id='TOP'></a>
# HOI Hub
Welcome to the SIT ICT3104 Team 5 Human Object Interaction Hub Juypter Notebook!
<br> This notebook will take you through the steps of running an "out-of-the-box" object detection model on video data.

Notebook is currently preloaded with these models:
1. [Toyota Smarthome (TSU) project](https://project.inria.fr/toyotasmarthome/)
2. [I3D Feature Extraction](https://github.com/Finspire13/pytorch-i3d-feature-extraction)
3. [TSU evaluation](https://github.com/dairui01/TSU_evaluation)
4. [Nvidia STEP: Spatio-Temporal Progressive Learning for Video Action Detection](https://github.com/NVlabs/STEP)
5. [MS-TCT: Multi-Scale Temporal ConvTransformer for Action Detection](https://github.com/dairui01/MS-TCT)
6. [I3D models trained on Kinetics](https://github.com/piergiaj/pytorch-i3d)

## Import libraries
Description: Starting with the base imports. This cell is used for importing relevant libraries for the project.

In [ ]:
# Upload File using ipyfilechooser library
from ipyfilechooser import FileChooser
from ipywidgets import interact, interactive, fixed, interact_manual
from tqdm.notebook import tqdm, trange
# Video Player
from IPython.display import Video, display, clear_output
import time 
# Get the root directory of the project
from pyprojroot import here
# Copy File
import shutil
# Widget Packages
import ipywidgets as widgets
# In case widget extension not working
# jupyter nbextension enable --py widge|tsnbextension
import os
import sys
import re
from dotenv import load_dotenv, find_dotenv
load_dotenv()
os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')
global_path = here("")

### Load Video Input Functions
Description: This cell is used for opening the file selector for user to select the video as input.

In [ ]:
def videoselectorinput():
    starting_directory = '../data/video'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser
def videoselectoroutputTSU():
    starting_directory = './video/output'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser

## Pipeline Selection
In this section, user can select the pipelines specified in this list to run the project:

- Toyota TSU (Toyota Smarthome Untrimmed)
- Nvidia STEP (Spatio-Temporal Progressive Learning for Video Action Detection)
- MS-TCT (Multi-Scale Temporal ConvTransformer for Action Detection)

Users can also add new pipelines.

### Adding new pipeline
To add a new pipeline, add in the name of the pipeline in the options list in the dropdown widgets and add in the code for the new pipeline in a folder at the root directory.

Description: This cell is used for setting and opening up the dropdown for the user to select the pipeline. After selecting and clicking "Confirm" the code will then run the selected pipeline.

In [ ]:
#print(os.getcwd())
pipelineList = ['TSU', 'STEP' , 'MS-TCT']
pipelineSelectedMessage = widgets.Label()
loadedDependenciesMessage = widgets.Label()
pipelineDropdown = widgets.Dropdown(
    options=pipelineList,
    value=pipelineList[0],
    description='Pipeline:')
# Function on what happen when confirm is been click.
def selectPipelineSet(b):
    if pipelineDropdown.value == "STEP":
        directory = os.getcwd()
        if re.search(r"TSU", directory):
            os.chdir('../')
            path =here("./NVIDIA-STEP-MODEL/STEP")
            os.chdir(path)
        elif re.search(r"MSTCT", directory):
            os.chdir('../')
            path =here("./NVIDIA-STEP-MODEL/STEP")
            os.chdir(path)
        elif re.search(r"STEP", directory):
            pass
        else:
            path =here("./NVIDIA-STEP-MODEL/STEP")
            os.chdir(path)
        pipelineSelectedMessage.value = "Running " + pipelineDropdown.value + " pipeline"
        loadedDependenciesMessage.value = "Loaded dependecies for " + pipelineDropdown.value
    elif pipelineDropdown.value == "TSU":
        directory = os.getcwd()
        if re.search(r"TSU", directory):
            pass
        elif re.search(r"STEP", directory):
            os.chdir('../../')
            path =here("./pipeline")
            os.chdir(path)
        elif re.search(r"MSTCT", directory):
            os.chdir('../')
            path =here("./pipeline")
            os.chdir(path)
        else:
            path =here("./pipeline")
            os.chdir(path)
        pipelineSelectedMessage.value = "Running " + pipelineDropdown.value + " pipeline"
        loadedDependenciesMessage.value = "Loaded dependecies for " + pipelineDropdown.value
    elif pipelineDropdown.value == "MS-TCT":
        directory = os.getcwd()
        if re.search(r"MSTCT", directory):
            pass
        elif re.search(r"STEP", directory):
            os.chdir('../../')
            path =here("./MSTCT")
            os.chdir(path)
        elif re.search(r"TSU", directory):
            os.chdir('../')
            path =here("./MSTCT")
            os.chdir(path)
        else:
            path =here("./MSTCT")
            os.chdir(path)
        pipelineSelectedMessage.value = "Running " + pipelineDropdown.value + " pipeline"
        loadedDependenciesMessage.value = "Loaded dependecies for " + pipelineDropdown.value
    else:
        pass
    
pipelineConfirm = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

pipelineConfirm.on_click(selectPipelineSet)
messagesBox =  widgets.VBox([loadedDependenciesMessage, pipelineSelectedMessage])
pipelineBox = widgets.VBox([widgets.HBox([pipelineDropdown, pipelineConfirm]), messagesBox])
pipelineBox

<a href="#TSU">Click here to go to Toyota TSU section</a><br>
<a href="#STEP">Click here to go to Nvidia STEP section</a><br>
<a href='#MS-TCT'>Click here to go to MS-TCT section</a>

---

# Toyota Smarthome (TSU)
This section is based on pretrained model from [Toyota Smarthome Project](https://github.com/dairui01/Toyota_Smarthome).
<br> With data sourced from [Toyota Smarthome (TSU) project](https://project.inria.fr/toyotasmarthome/).

## Data Exploration Section
This section in the notebook that can load and display video data from the Toyota Smarthome (TSU) project.

### Video Upload / Choose using ipyfilechooser
Description: This cell is used for setting up the file choosers to select input video and to save output video.

In [ ]:
video = videoselectorinput()

### Upload selected video to the data folder (If needed)
Description: This cell is used for uploading the video to data folder if video isn't already inside.

In [ ]:
# Upload Function
# from pyprojroot import here
# import shutil
def upload(video):
    print(video.selected)
    source = video.selected
    # Source path
    # Destination path
    destination = (here("./data/video"))

    # Copy file from the selected path
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")

    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If destination is a directory.
    except IsADirectoryError:
        print("Destination is a directory.")

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")

upload(video)

### Video Input Selection (Load video)
Description: This cell is used for opening the file selector for user to select video input.

In [ ]:
# Select Video
video = videoselectorinput()

### Video Player (Play selected video)
Description: This cell is used for running the input video. This is for the user to check if it is the same video they selected.

In [ ]:
full_path = video.selected
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

<a id='TSU'></a>
## Inference Section
This section in the notebook that can perform inference using a pretrained HOI ML model based on the TSU project.

### Load a pretrain model
Description: This cell is used for setting up the dropdown list to allow the user to select the model to load.

In [ ]:
# Select model
modelList = [] 
modelSelectedMessage = widgets.Label()

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")) == False:
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    modelSelectedMessage.value = "Selected: " + modelDropdown.value
    #print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton]), modelSelectedMessage])
modelBox

### Choose Input video to load into TSU model
Description: This cell is used for selecting the video input to be passed to the TSU model.
<br> Note: You only can select video that is in **testing subset** on the smarthome_CS_51.json file to run the inference video.

In [ ]:
# Select Video
video = videoselectorinput()

### Run inference on the TSU model
Description: This cell is used for running the inference.py script based on the video and model selected.

In [ ]:
model = modelDropdown.value
loadmodel = './models/' + model
videoPath = video.selected
videoFile = video.selected_filename
print(videoFile)
print(videoPath)
%run -it inference.py  -input_video_file $videoFile -model $model  -load_model $loadmodel -video_path $videoPath
%wandb ict3104-team-5/inference-visualisation

### Output Video to view the inference result
Description: This cell is used for print the filepath of the output video. Play the output video below the cell.

In [ ]:
videoFileName = video.selected_filename[:-4]
full_path = (os.path.join(here("./pipeline/video/output/"),f"{videoFileName}_caption.mp4"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

## Feature Extraction Section
This section in the notebook generates feature files from videos and validates the smarthome_CS_51.json to be used for training.

### Run main feature-extraction function
Description: This cell is used for running the main feature-extraction function to extract features from videos listed in video_paths.txt to create RGB .npy files for model training.

In [ ]:
path = os.path.join(global_path , 'i3d-feature-extraction') 
%cd $path
%run -it main.py feature_type=i3d device="cuda:0" on_extraction=save_numpy streams=rgb output_path=../data/dataset/v_iashin_i3d/ file_with_video_paths=./sample/sample_video_paths.txt

### Remove entries that do not have RGB .npy files
Description: This cell is used for running validate_train_test.py to  remove entries that do not have RGB .npy files generated in a specific directory from smarthome_CS_51.json to create an updated one called smarthome_CS_51_v2.json.

In [ ]:
fc = FileChooser(f'{global_path}/data/dataset') 
fc.show_only_dirs = True
fc.title = 'Select your RGB .npy files directory'

def update_my_fname():
    global my_fname
    my_fname = fc.selected
    return  

fc.register_callback(update_my_fname)

ratio_list = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
        
def dropdown_eventhandler(change):
    determine(training_ratio_dropdown.value)

def determine(ratio):
    testing_ratio_dropdown.value = 100 - ratio

training_ratio_dropdown = widgets.Dropdown(description="Training ratio:", options=ratio_list)
testing_ratio_dropdown = widgets.Dropdown(description="Testing ratio:", options=ratio_list, disabled=True)

training_ratio_dropdown.observe(dropdown_eventhandler, names='value')

button = widgets.Button(
    description='Run',
    disabled=False,
    button_style='success'
)

display(fc, training_ratio_dropdown, testing_ratio_dropdown, button)

def run_validation(b):
    relative_path = os.path.relpath(my_fname, path)
    
    training_ratio = training_ratio_dropdown.value
    testing_ratio = testing_ratio_dropdown.value
    
    print(f"RGB .npy files directory: {relative_path}")
    print(f"Training ratio: {training_ratio}")
    print(f"Testing ratio: {testing_ratio}")
    print("\n")
    
    !python validate_train_test.py -rgb_path $relative_path -training_ratio $training_ratio -testing_ratio $testing_ratio
    
button.on_click(run_validation)


## Training Section
This section in the notebook that can train a HOI ML model based on the TSU project.

### Run model training function
Description: This cell is used for form to fill up the parameters needed to run the run_PDAN.sh for training. Running the run_PDAN.sh script based on the parameters.

In [ ]:
# Prints working Directory
%pwd

In [ ]:
path = os.path.join(global_path , 'pipeline') 
os.chdir(path)
title = widgets.HTML("<h1 style=\"text-align: center; \"><b>Run_PDAN</b></h1>")
label = widgets.HTML(
    value="<b>Label</b>\
    <br><b>Dataset Input:</b> Name of the Dataset to train\
    <br><b>Mode:</b> Inference Metric Methods: rgb or flow\
    <br><b>Spilt Setting:</b> Either CS or CV\
    <br><b>Model:</b> Select Model To Train\
    <br><b>Train:</b> Train or Test\
    <br><b>Num Channel:</b> Number of channel for image processing\
    <br><b>Learning Rate:</b> Learning rate indicates the step size that gradient descent takes towards local optima\
    <br><b>Kernel Size:</b> Size of a convolutional filter\
    <br><b>AP Type:</b> Metric in measuring the accuracy of object detectors: map or wap\
    <br><b>Epoch:</b> Number of iterations for the training process in Machine Learning\
    <br><b>Batch_Size:</b> Number of samples that is used in one epoch.\
    <br><b>Compute Info:</b> Information of the computation"
)
style = {'description_width': '90px'}

# Using Dropdown method
dataset_list = []
directoryDataset = os.path.join(global_path , 'data' , 'dataset') # Directory of the dataset (NPY files)
# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    value=dataset_list[0],
    style=style,
    description='Dataset:')


mode_input = widgets.Text(
    value='rgb',
    placeholder='Enter Mode',
    description='Mode:',
    disabled=False,
    style=style,
    tooltip='Inference Metric Methods: rgb or flow'
    
)

split_input = widgets.Text(
    value='CS',
    placeholder='Enter Split Setting',
    description='Split Setting:',
    disabled=False,
    style=style,
    tooltip='Either CS or CV'
)

# Using Dropdown method
model_list = []
model_list.append("PDAN_TSU_RGB")
        
model_input = widgets.Dropdown(
    options=model_list,
    value=model_list[0],
    description='Model:',
    style=style,
    tooltip='Select Model to train.'
)


train_input = widgets.Checkbox(
    value=True,
    description = 'Train',
    disabled=False,
    indent=True,
    style=style,
    tooltip='Train or Test'
)

num_channel_input =  widgets.BoundedIntText(
    value=512,
    min=1,
    max=1000,
    step=1,
    description='Num Channel:',
    disabled=False,
    style=style,
    tooltip='Number of channel for image processing'
)

lr_input = widgets.FloatText(
    value=0.0002,
    description='Learning Rate:',
    disabled=False,
    style=style,
    tooltip='Learning rate indicates the step size that gradient descent takes towards local optima'
)

kernel_size_input =  widgets.BoundedIntText(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Kernel Size:',
    disabled=False,
    style=style,
    tooltip='Size of a convolutional filter'
)

aptype_input = widgets.Text(
    value='map',
    placeholder='Enter APType',
    description='APType:',
    disabled=False,
    style=style,
    tooltip='Metric in measuring the accuracy of object detectors: map or wap'
)

epoch_input = widgets.BoundedIntText(
    value=140,
    min=1,
    max=1000,
    step=1,
    description='Epoch:',
    disabled=False,
    style=style,
    tooltip='Number of iterations for the training process in Machine Learning'
)

batch_size_input = widgets.Dropdown(
    options=['1', '2', '4', '8', '16', '32', '64', '128', '256', '512', '1024'],
    value='1',
    description='Batch_Size:',
    disabled=False,
    tooltip='Number of samples that is used in one epoch.',
    style=style
    
)


comp_info_input = widgets.Text(
    value='TSU_CS_RGB_PDAN',
    placeholder='Enter Compute Info',
    description='Compute Info:',
    disabled=False,
    style=style,
    tooltip='Information of the computation.'
)

button = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Pref',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    style=style
)

jsonDirectory = os.path.join(directoryDataset , 'JSON')
jsonFiles = []
for x in os.listdir(jsonDirectory):
    if os.path.isdir(os.path.join(jsonDirectory, x)) == False:
        jsonFiles.append(x)

json_file_input = widgets.Dropdown(
    options=jsonFiles,
    value=jsonFiles[0],
    style=style,
    description='Json File:')
    

def selectWidgetSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
    print("Selected Mode: " , mode_input.value)
    print("Selected Split Setting: " , split_input.value)
    print("Selected Model: " , model_input.value)
    print("Selected Train: " , train_input.value)
    print("Selected Num Channel: " , num_channel_input.value)
    print("Selected Learning Rate: " , lr_input.value)
    print("Selected Kernel Size: " , kernel_size_input.value)
    print("Selected APType: ", aptype_input.value)
    print("Selected Epoch: ", epoch_input.value)
    print("Selected Batch: " , batch_size_input.value)
    print("Selected Compute Info: " , comp_info_input.value)
    dataset_path = os.path.join("../data/dataset/" , datasetDropDown.value)
    print("Dataset Path: ", dataset_path)
    json_path = os.path.join("../data/dataset/JSON/" , json_file_input.value)
    print("Json Path: ", json_path)
    %run -i train.py -dataset $datasetDropDown.value -mode \
    $mode_input.value -split_setting $split_input.value -model $model_input.value \
    -train $train_input.value -num_channel $num_channel_input.value -lr $lr_input.value \
    -kernelsize $kernel_size_input.value -APtype $aptype_input.value \
    -epoch $epoch_input.value -batch_size $batch_size_input.value -comp_info $comp_info_input.value \
    -rgb_root $dataset_path -json_file $json_path
    %wandb ict3104-team-5/training-visualisation

display(title, label, datasetDropDown, mode_input,split_input, model_input, train_input, num_channel_input, lr_input, kernel_size_input, aptype_input, epoch_input, batch_size_input, comp_info_input, json_file_input ,button)
button.on_click(selectWidgetSet)

## Testing Section
This section in the notebook that will evaluate a trained model based on the TSU project.

### Model selection for Testing
Description: This cell is used for dropdown for the user to select the model to be tested.

In [ ]:
path = os.path.join(global_path , 'pipeline') 
os.chdir(path)
# Select model
modelList = []
modelSelectedTestingMessage = widgets.Label() 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")):
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    modelSelectedTestingMessage.value = "Selected: " + modelDropdown.value
    %run -i test.py -split CS -pkl_path "models/"$modelDropdown.value
    %wandb ict3104-team-5/testing-visualisation

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton]), modelSelectedTestingMessage])
modelBox

### Inference Section for Training model
Description: This cell runs inference on the training model selected.

In [ ]:
# Select model
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./models"): 
    if os.path.isdir(os.path.join("./models", x)) == False:
        if (x.endswith(".pkl")) == False:
            modelList.append(x)

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)
    model = modelDropdown.value
    loadmodel = './models/' + model
    videoPath = video.selected
    videoFile = video.selected_filename
    print(videoFile)
    print(videoPath)
    %run -it inference.py  -input_video_file $videoFile -model $model  -load_model $loadmodel -video_path $videoPath
    %wandb ict3104-team-5/inference-visualisation

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

### Output Video for Inference Section for Training model
Description: This cell runs inference on the training model selected.

In [ ]:
videoFileName = video.selected_filename[:-4]
full_path = (os.path.join(here("./pipeline/video/output/"),f"{videoFileName}_caption.mp4"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

---

<a id='STEP'></a>
# NVIDIA STEP Section
## Nvidia Step Pipeline

**Description:** This cell is used for setup the model

In [ ]:
#Run this cell to setup the NVIDIA STEP.
%run -i setup.py build develop

<a href="#INFERENCE">Click here to go to NVIDIA STEP INFERENCE Section</a><br>
<a href="#TRAINING">Click here to go to NVIDIA STEP TRAINING Section</a><br>
<a href="#TESTING">Click here to go to NVIDIA STEP TESTING Section</a><br>

<a id='INFERENCE'></a>
## INFERENCE

**Description:** This cell is used for selection of the dataset and the video input to run INFERENCE

In [ ]:
# Select video
video_list = [] 
dataset_list = [] 
model_list = [] 
directoryVideo = "./Input/" # Directory of the video (.mp4)
directoryDataset = "./datasets/Inference/frames/" # Directory of the dataset (NPY files)
directoryModel = "./pretrained/" # Directory of the pretrained model

datasetStepSelectedMessage = widgets.Label()
videoStepSelectedMessage = widgets.Label()
# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

# Store the video names into video list
for x in os.listdir(directoryVideo):
    if x.endswith(".mp4"):
        video_list.append(x)
        
# Store the pretained model names into pretrained model list
for x in os.listdir(directoryModel):
    if x.endswith(".pth"):
        model_list.append(x)
        
datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    description='Dataset:')

videoDropDown = widgets.Dropdown(
    options=video_list,
    description='Video: ')

modelDropDown = widgets.Dropdown(
    options=model_list,
    description='Model: ')

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
# Function on what happen when confirm is been click. To be intergrated with some other code
def selectTrimSet(b):
    datasetStepSelectedMessage.value = "Selected Dataset: " + datasetDropDown.value
    videoStepSelectedMessage.value = "Selected Video: " + videoDropDown.value
    #print("Selected Dataset: " , datasetDropDown.value)
    #print("Selected Video: " , videoDropDown.value)
    print("Selected model: " , modelDropDown.value)

confirmButton.on_click(selectTrimSet)
display(datasetDropDown, videoDropDown, modelDropDown, confirmButton, datasetStepSelectedMessage, videoStepSelectedMessage)

**Description:** This cell is used to caption every frame of the video. Afterwards, it will run the STEP Model.<br>
**Note:** The root folder selected will be use in this section to store the frames generated from the video selected in the frames folder within the root folder.<br> *** Output frames result from the INFERENCE will be store in the Output folder.

In [ ]:
import cv2
import numpy as np
import glob
cap= cv2.VideoCapture('Input/' + videoDropDown.value)
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('datasets/Inference/frames/' + datasetDropDown.value + '/frame'+str(i).zfill(4)+'.jpg',frame)
    print('frame generated: ' + str(i).zfill(4) + '.jpg')
    i+=1

cap.release()
cv2.destroyAllWindows()


data_root="datasets/Inference/"
%run -i demo.py -model $modelDropDown.value -root_folder $data_root

**Description:** This cell is used for reading every frames in the results folder.It will write the captions of the frames onto the output video.

In [ ]:
import cv2
import numpy as np
import glob

img_array = []
for filename in glob.glob('Output/results/' + datasetDropDown.value + '/*.jpg'):
    img = cv2.imread(filename) 
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
out = cv2.VideoWriter('Output/'+videoDropDown.value, fourcc, 29, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

vdoFileName = videoDropDown.value
full_path = (os.path.join(here("./Output/"),f"{vdoFileName}"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

<a id='TRAINING'></a>
## TRAINING

**Description:** The cell will generate pickle files (.PKL) from the csv files prepared in "datasets/ava/label/"<br>
**Please change the content in the csv files according to the name of the of the video you downloaded from AVA dataset** <br>
**Note:** The current csv files, each contain one video name in the first column. You can look for more information of NVIDIA STEP <a href="https://github.com/NVlabs/STEP">here</a>.<br>
Download AVA dataset version 2.1 <a href="https://research.google.com/ava/download/ava_v2.1.zip">here</a>

In [ ]:
import torch
import cv2
%run ./scripts/generate_label.py "datasets/ava/label/ava_train_v2.1.csv" 
%run ./scripts/generate_label.py "datasets/ava/label/ava_val_v2.1.csv"

**Description:** This cell will extract frames out from the video input <br>
**To get the video for training using AVA dataset, please go to https://github.com/cvdfoundation/ava-dataset to download the video**

In [ ]:
%run ./scripts/extract_clips.py

**Description:** Customize the setting to run the training. For device with low resources recommended to set **num_workers to 2 and batch_size to 1 or both at 1**

In [ ]:
data_root="datasets/ava/"
save_root="datasets/ava/cache/"
pretrain_path="pretrained/ava_step.pth"

name="STEP"
base_net="i3d"
det_net="two_branch"
resume_path="Auto"

T=3
max_iter=3    # index starts from 1
iterative_mode="temporal"
anchor_mode="1"
temporal_mode="predict"
pool_mode="align"
pool_size=7

# training schedule
num_workers=2
max_epochs=144
batch_size=1
optimizer="adam"
base_lr=7.5e-5
det_lr0=1.5e-4
det_lr=7.5e-4
save_step=11465
print_step=500
scheduler="cosine"
milestones="-1"
warmup_iters=1000

# losses
dropout=0.3
fc_dim=256
lambda_reg=5
lambda_neighbor=1
cls_thresh="0.2,0.35,0.5"
reg_thresh="0.2,0.35,0.5"
max_pos_num=5
neg_ratio=2
NUM_SAMPLE=-1
topk=300
evaluate_topk=300

# data augmentation / normalization
scale_norm=2    # for i3d
do_flip="True"
do_crop="True"
do_photometric="True"
do_erase="True"
freeze_affine="True"
freeze_stats="True"


%run train.py --data_root $data_root --save_root $save_root \
    --name $name --pretrain_path $pretrain_path --resume_path $resume_path \
    --base_net $base_net --det_net $det_net --max_iter $max_iter --T $T \
    --iterative_mode $iterative_mode --anchor_mode $anchor_mode --anchor_mode $anchor_mode --temporal_mode $temporal_mode \
    --pool_mode $pool_mode --pool_size $pool_size --save_step $save_step --topk $topk --evaluate_topk $evaluate_topk \
    --num_workers $num_workers --max_epochs $max_epochs --batch_size $batch_size --print_step $print_step \
    --optimizer $optimizer --base_lr $base_lr --det_lr $det_lr --det_lr0 $det_lr0 --milestones $milestones \
    --scale_norm $scale_norm --do_flip $do_flip --do_crop $do_crop --do_photometric $do_photometric --do_erase $do_erase \
    --fc_dim $fc_dim --dropout $dropout --NUM_SAMPLE $NUM_SAMPLE --scheduler $scheduler --warmup_iters $warmup_iters \
    --cls_thresh $cls_thresh --reg_thresh $reg_thresh --max_pos_num $max_pos_num --neg_ratio $neg_ratio \
    --freeze_affine $freeze_affine --freeze_stats $freeze_stats --lambda_reg $lambda_reg --lambda_neighbor $lambda_neighbor

**Description:** This cell will copy all the file with ".pth" extention to "./pretrained/" folder

In [ ]:
from pathlib import Path
import shutil

src_path = 'datasets/ava/cache/STEP-max3-i3d-two_branch/'
trg_path = 'pretrained/'

for src_file in Path(src_path).glob('*.pth'):
    shutil.copy(src_file, trg_path)

<a id='TESTING'></a>
## Testing

**Description:** This cell is to run the testing for NVIDIA STEP model <br>
Run the following command for testing and evaluation on the validation set of AVA <br>
**Note:** The output will be stored in **"datasets/ava/cache/STEP-max3-i3d-two_branch/"**

In [ ]:
# Select model
model_list = [] 
directoryModel = "./pretrained/" # Directory of the pretrained model

# Store the pretained model names into pretrained model list
for x in os.listdir(directoryModel):
    if x.endswith(".pth"):
        model_list.append(x)      

modelDropDown = widgets.Dropdown(
    options=model_list,
    description='Model: ')

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
# Function on what happen when confirm is been click. To be intergrated with some other code
def selectTrimSet(b):
    print("Selected model: " , modelDropDown.value)

confirmButton.on_click(selectTrimSet)
print("Selected pretrained model for testing.")
display(modelDropDown, confirmButton)

In [ ]:
%run test.py -model $modelDropDown.value

<a id='MS-TCT'></a>
# MS-TCT Section
T05-173: As a user, I want to set up the environment for MS-TCT, so that I am able to run the base code <br>
<a href="#TOP">Click here to go back to the import library section</a>

## PyTorch I3D Feature Extraction
In this section we will be extracting the npy file for the I3D model. Select the dataset folder to extract the features.

In [ ]:
dataset_list = []
directoryDataset = os.path.join(global_path , 'data' , 'dataset') # Directory of the dataset (NPY files)
# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)
datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    value=dataset_list[0],
    description='Dataset:')
# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
def selectDataset(b):
    print("Selected dataset: " + datasetDropDown.value)
confirmButton.on_click(selectDataset)
dataBox = widgets.VBox([widgets.HBox([datasetDropDown, confirmButton])])
dataBox

Select folder to save the feature extracted.

In [ ]:
def resultPyTorchI3D():
    starting_directory = './data'
    chooser = FileChooser(starting_directory)
    chooser.show_only_dirs = True
    display(chooser)
    return chooser
result = resultPyTorchI3D()

In [ ]:
%cd ../pytorch-i3d-master
dataset_path  = os.path.join("../data/dataset/" , datasetDropDown.value)
print("Dataset Path: ", dataset_path)
print("Result Path: " , result.selected)
result_path = result.selected
start_path = os.getcwd()
relative_path = os.path.relpath(result_path, start_path)
# Convert \\ to / for relative path
for i in range(len(relative_path)):
    if relative_path[i] == '\\':
        relative_path = relative_path[:i] + '/' + relative_path[i+1:]
print(relative_path)
%run -it extract_features.py -mode rgb -root ../data/dataset/Charades -gpu 0 -save_dir $relative_path -load_model ./models/rgb_charades.pt
%cd ../MSTCT

## Training Section
In this section we will be training the model using the extracted features.

In [ ]:
title = widgets.HTML("<h1 style=\"text-align: center; \"><b>Run_MSTCT_Charades</b></h1>")
label = widgets.HTML(
    value="<b>Label</b>\
    <br><b>Dataset Input:</b> Name of the Dataset to train\
    <br><b>Mode:</b> Inference Metric Methods: rgb or flow\
    <br><b>Model:</b> Select Model To Train\
    <br><b>Train:</b> Train or Test\
    <br><b>Num Clips:</b> Number of Clips for image processing\
    <br><b>Learning Rate:</b> Learning rate indicates the step size that gradient descent takes towards local optima\
    <br><b>Skip:</b> Skip\
    <br><b>Alpha Learning Rate:</b> Alpha Learning Rate\
    <br><b>Beta Learning Rate:</b> Beta Learning Rate\
    <br><b>Epoch:</b> Number of iterations for the training process in Machine Learning\
    <br><b>Batch_Size:</b> Number of samples that is used in one epoch.\
    <br><b>Compute Info:</b> Information of the computation \
    <br><b>Uni Size:</b> Uni Size"
)
style = {'description_width': '90px'}

# Using Dropdown method
dataset_list = []
directoryDataset = os.path.join(global_path , 'data' , 'dataset') # Directory of the dataset (NPY files)
# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    value=dataset_list[0],
    style=style,
    description='Dataset:')


mode_input = widgets.Text(
    value='rgb',
    placeholder='Enter Mode',
    description='Mode:',
    disabled=False,
    style=style,
    tooltip='Inference Metric Methods: rgb or flow'
    
)


# Using Dropdown method
model_list = []
model_list.append("MS_TCT")
        
model_input = widgets.Dropdown(
    options=model_list,
    value=model_list[0],
    description='Model:',
    style=style,
    tooltip='Select Model to train.'
)

train_input = widgets.Checkbox(
    value=True,
    description = 'Train',
    disabled=False,
    indent=True,
    style=style,
    tooltip='Train or Test'
)


num_clips_input =  widgets.BoundedIntText(
    value=256,
    min=1,
    max=1000,
    step=1,
    description='Num Clips:',
    disabled=False,
    style=style,
    tooltip='Number of clips for image processing'
)

skip_input = widgets.BoundedIntText(
    value=0,
    placeholder='Enter Skip',
    description='Skip Setting:',
    disabled=False,
    style=style,
    tooltip='Skip Setting'
)


lr_input = widgets.FloatText(
    value=0.0001,
    description='Learning Rate:',
    disabled=False,
    style=style,
    tooltip='Learning rate indicates the step size that gradient descent takes towards local optima'
)


comp_info_input = widgets.Checkbox(
    value=False,
    description = 'Compute Info',
    disabled=False,
    indent=True,
    style=style,
    tooltip='True or False'
)


epoch_input = widgets.BoundedIntText(
    value=50,
    min=1,
    max=1000,
    step=1,
    description='Epoch:',
    disabled=False,
    style=style,
    tooltip='Number of iterations for the training process in Machine Learning'
)


unisize_input =  widgets.Checkbox(
    value=True,
    description = 'Unisize',
    disabled=False,
    indent=True,
    style=style,
    tooltip='True Or False'
)

alpha_l_input = widgets.FloatText(
    value=1,
    description='Alpha l:',
    disabled=False,
    style=style,
    tooltip='Learning rate for Alpha'
)

beta_l_input = widgets.FloatText(
    value=0.05,
    description='Beta l:',
    disabled=False,
    style=style,
    tooltip='Learning rate for Beta'
)

batch_size_input = widgets.Dropdown(
    options=['1', '2', '4', '8', '16', '32', '64', '128', '256', '512', '1024'],
    value='1',
    description='Batch_Size:',
    disabled=False,
    tooltip='Number of samples that is used in one epoch.',
    style=style
    
)

button = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Pref',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    style=style
)

jsonDirectory = os.path.join(directoryDataset , 'JSON')
jsonFiles = []
for x in os.listdir(jsonDirectory):
    if os.path.isdir(os.path.join(jsonDirectory, x)) == False:
        jsonFiles.append(x)

json_file_input = widgets.Dropdown(
    options=jsonFiles,
    value=jsonFiles[0],
    style=style,
    description='Json File:')
    

def selectWidgetSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
    print("Selected Mode: " , mode_input.value)
    print("Selected Model: " , model_input.value)
    print("Selected Train: " , train_input.value)
    print("Selected Num Clips: " , num_clips_input.value)
    print("Selected Skip: " , skip_input.value)
    print("Selected Learning Rate: " , lr_input.value)
    print("Selected Compute Info: " , comp_info_input.value)
    print("Selected Epoch: ", epoch_input.value)
    print("Selected Uni Size: " , unisize_input.value)
    print("Selected Alphal: ", alpha_l_input.value)
    print("Selected Betal: ", beta_l_input.value)
    print("Selected Batch: " , batch_size_input.value)
    dataset_path = os.path.join("./data/" , datasetDropDown.value)
    print("Dataset Path: ", dataset_path)
    output_path = os.path.join("./save_logit/")
    # delete(output_path , '.pkl')
    %run -it train.py -dataset $datasetDropDown.value \
    -mode $mode_input.value -model $model_input.value \
    -train $train_input.value -num_clips $num_clips_input.value -skip $skip_input.value \
    -lr $lr_input.value -comp_info $comp_info_input.value -epoch $epoch_input.value \
    -unisize $unisize_input.value -batch_size $batch_size_input.value -rgb_root $dataset_path

def delete(dir, pattern):
    for f in os.listdir(dir):
        if re.search(pattern, f):
            os.remove(os.path.join(dir, f))

display(title, label, datasetDropDown, mode_input, model_input, train_input, num_clips_input, skip_input, lr_input, comp_info_input, epoch_input, unisize_input, alpha_l_input,beta_l_input, batch_size_input, button)
button.on_click(selectWidgetSet)

## Evaluation 
In this section we will be evulating the trained model.

In [ ]:
# Select model
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./save_logit"): 
    if os.path.isdir(os.path.join("./save_logit", x)) == False:
        if (x.endswith(".pkl")):
            modelList.append(x)

modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:'
    )

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
    )
 
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)
    model_path = os.path.join("./save_logit/" , modelDropdown.value)
    print("Model Path: ", model_path)
    %run -i Evaluation.py -pkl_path $model_path

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox